# Job Placement Predictor

This project builds a model that predicts whether a student can get job 
placement opportunity based on his/her academic performance, work experiences, projects etc. 

## Project Work Flow

1. Load the data into a dataframe
2. Perform Data Preprocessing like handling missing values, feature creation etc.
3. Perform Eploratory Data Analysis and get valuable insights from the data
4. Perform feature selection and select the best algorithm which fits the data
5. Save the model in a pickle file

In [ ]:
# import libriaries
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt